In [82]:
import numpy as np
import tools
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from matplotlib import pyplot as plt
from metrics import Metrics
# Some magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
lan_codes = ['en', 'es', 'pt', 'gl', 'eu', 'ca', 'fr', 'it', 'de']
Languages = ['English', 'Spanish', 'Portuguese', 'Galician', 'Basque', 'Catalan', 'French', 'Italian', 'German']

## Load the Data, Calculate Vocabulary and Perplexity

In [84]:
# Load Data
data = tools.Data()
print('Size of the vocabulary: %d characters' % len(data.vocab))

Size of the vocabulary: 509 characters


In [85]:
preplexity = data.get_perplexity()
print('Perplexity measurement is %.2f' % preplexity)

Perplexity measurement is 34.11


In [86]:
print('Percent of Invalid Characters - Train: %.5f%%' 
    % ((data.train_freq[data.vocab.index('<N>')] / data.train_freq.sum()) * 100.0))
print('Percent of Invalid Characters - Val: %.5f%%' 
    % ((data.val_freq[data.vocab.index('<N>')] / data.val_freq.sum()) * 100.0))

Percent of Invalid Characters - Train: 0.04622%
Percent of Invalid Characters - Val: 0.05987%


## Model

In [87]:
# this cell takes a few minutes to run
train_tweets, train_lans = tools.data_encoding(data.train, data.vocab, lan_codes)
val_tweets, val_lans = tools.data_encoding(data.val, data.vocab, lan_codes)
test_tweets, test_lans = tools.data_encoding(data.test, data.vocab, lan_codes)

In [88]:
class MyRNN(nn.Module):
    def __init__(self, vocab_size, lang_size=9, char_vec_size=12, lang_vec_size=2, hidden_size=50, PAD=0):
        super(MyRNN, self).__init__()
        self.vocab_size = vocab_size
        self.lang_size = lang_size
        self.char_vec_size = char_vec_size
        self.lang_vec_size = lang_vec_size
        self.hidden_size = hidden_size
        
        self.char_encoder = nn.Embedding(self.vocab_size, self.char_vec_size)
        self.lang_encoder = nn.Embedding(self.lang_size, self.lang_vec_size)
        # the current hidden size = char_vec_size
        self.gru = nn.GRU(self.char_vec_size+self.lang_vec_size, self.hidden_size, num_layers=1)
        self.linear = nn.Linear(self.hidden_size, self.char_vec_size)
        self.decoder = nn.Linear(self.char_vec_size, self.vocab_size)
        
        # This shares the encoder and decoder weights as described in lecture.
        self.decoder.weight = self.char_encoder.weight
        self.decoder.bias.data.zero_()
        
        
        weight = torch.ones(vocab_size)
        # scores over PAD is not counted
        weight[PAD] = 0
        self.sm = nn.LogSoftmax(dim=1)
        self.crit = nn.NLLLoss(weight, size_average=False)

    def forward(self, tweets, lang, hidden=None):
        emb = torch.cat((self.lang_encoder(lang), self.char_encoder(tweets)), -1)
        output, hidden_t = self.gru(emb, hidden)
        output = F.tanh(self.linear(output))
        output = self.decoder(output)
        return output, hidden_t

    # Predefined loss function
    def loss(self, prediction, label, reduction='elementwise_mean'):
        prediction = prediction.view(-1, self.vocab_size)
        prediction = self.sm(prediction)
        loss_val = self.crit(prediction, label.view(-1))
        return loss_val

In [89]:
BATCH_SIZE = 64
FEATURE_SIZE = 15
TEST_BATCH_SIZE = 256
EPOCHS = 2
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0005
MOMENTUM = 0.5
LOG_INTERVAL = 500
device = torch.device("cpu")

In [110]:
model = MyRNN(len(data.vocab), PAD=data.vocab.index('</S>'))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [92]:
tools.train(model, device, train_loader, optimizer, EPOCHS, LOG_INTERVAL,verbose=True)

Train Epoch: 0 [0/80175 (0%)]	Loss: 75423.132812
Train Epoch: 0 [32000/80175 (40%)]	Loss: 1229.884888
Train Epoch: 0 [64000/80175 (80%)]	Loss: 651.479797
Train Epoch: 1 [0/80175 (0%)]	Loss: 325.552063
Train Epoch: 1 [32000/80175 (40%)]	Loss: 160.575851
Train Epoch: 1 [64000/80175 (80%)]	Loss: 173.832031
Train Epoch: 2 [0/80175 (0%)]	Loss: 66.720955
Train Epoch: 2 [32000/80175 (40%)]	Loss: 36.463741
Train Epoch: 2 [64000/80175 (80%)]	Loss: 29.066631


In [103]:
tools.test(model, device, val_loader, data.vocab.index('</S>'))

Perplexity : 184.42515049270366
Loss       : 5314.205371856689


(5314.205371856689, 184.42515049270366)

In [114]:
tweets_test = torch.tensor(test_tweets, dtype=torch.long, device=torch.device("cpu"))
language_val = torch.tensor(test_lans, dtype=torch.long, device=torch.device("cpu"))

In [115]:
pred = tools.predict(model, device, tweets_test[:5000,:])

In [116]:
print(f'Percent Correct: {np.sum(pred == test_lans[:5000,0])/pred.shape[0]*100}')

Percent Correct: 30.72


In [124]:
Metrics(pred, test_lans[:5000,0])

accuracy = 0.307
 Lang     Prec.   Rec.   F1
------------------------------
     0.0   31.70  97.02  47.79
     1.0    0.00   0.00   0.00
     2.0    0.00   0.00   0.00
     3.0    0.00   0.00   0.00
     4.0    0.00   0.00   0.00
     5.0    0.00   0.00   0.00
     6.0    4.44   3.54   3.94
     7.0    0.00   0.00   0.00
     8.0    0.00   0.00   0.00
     9.0    0.00   0.00   0.00
    10.0    0.00   0.00   0.00
    11.0    0.00   0.00   0.00
    12.0    0.00   0.00   0.00
    13.0    0.00   0.00   0.00
    14.0    0.00   0.00   0.00
------------------------------
  Total:    2.41   6.70   3.45


3.448548036938752